# CS246 - Colab 1
## Word Count in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=04e4868a288b9df6c35573daa4d8d3485b7837c4b483a26d52fb0f83f003731b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 19 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you successfully run the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter, we want to count the total number of (non-unique) words that start with a specific letter.

In your implementation, **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all words that **start** with a non-alphabetic character. You should output word counts for the **entire document**, inclusive of the title, author, and the main texts. If you encounter words broken as a result of new lines, e.g. "pro-ject" where the segment after the dash sign is on a new line, no special processing is needed and you can safely consider it as two words.

Your outputs will be graded on a range -- if your differences from the ground-truths are within an error threshold of 5, you'll be considered correct.

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import re

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [6]:
# YOUR
def normalize_text(text):
  text = text[0]
  return re.compile(r'\W+', re.UNICODE).split(text.lower())

txt_raw = spark.read.text("pg100.txt")
txt_raw.show()
# txt = txt.rdd.flatMap(lambda x: re.split('\\W', x[0])).filter(lambda x: x != '')
txt = txt_raw.rdd.flatMap(normalize_text).filter(lambda x: x != '')
txt.take(3)

+--------------------+
|               value|
+--------------------+
|The Project Guten...|
| William Shakespeare|
|                    |
|This eBook is for...|
|almost no restric...|
|re-use it under t...|
|with this eBook o...|
|                    |
|** This is a COPY...|
|**     Please fol...|
|                    |
|Title: The Comple...|
|                    |
|Author: William S...|
|                    |
|Posting Date: Sep...|
|Release Date: Jan...|
|                    |
|   Language: English|
|                    |
+--------------------+
only showing top 20 rows



['the', 'project', 'gutenberg']

In [7]:
# CODE
import pyspark.sql.functions as F
# txt = txt.map(lambda x: (F.upper(F.substr(x, 1, 2)), 1))
txt = txt.map(lambda x: (x[0].upper(), 1))
txt.take(3)

[('T', 1), ('P', 1), ('G', 1)]

In [8]:
# HERE
result = txt.reduceByKey(lambda x, y: x + y).sortBy(lambda v: v[1]).collect()
result

[('_', 2),
 ('0', 10),
 ('X', 14),
 ('8', 21),
 ('7', 21),
 ('6', 26),
 ('9', 32),
 ('5', 39),
 ('4', 49),
 ('3', 69),
 ('Z', 79),
 ('2', 325),
 ('1', 917),
 ('Q', 2388),
 ('J', 3372),
 ('V', 5801),
 ('U', 9230),
 ('K', 9535),
 ('R', 15234),
 ('E', 20409),
 ('G', 21167),
 ('Y', 25926),
 ('N', 27313),
 ('P', 28059),
 ('L', 32389),
 ('C', 34983),
 ('F', 37186),
 ('D', 39173),
 ('O', 43712),
 ('B', 46001),
 ('M', 56252),
 ('W', 60097),
 ('H', 61028),
 ('I', 62420),
 ('S', 75226),
 ('A', 86000),
 ('T', 127781)]

Same thing with Spark DataFrames:

In [16]:
import pyspark.sql.functions as F

txt_words = txt_raw.select(F.explode(F.split(txt_raw.value, '\\W+')).alias('word'))
txt_words.show()
txt_words = txt_words.filter(txt_words.word != '')
txt_words = txt_words.select(F.substring(F.upper(txt_words.word), 1, 1).alias('starting_letter'))
letter_counts = txt_words.groupBy('starting_letter').count().sort('count')
letter_counts.show(50)

+-----------+
|       word|
+-----------+
|        The|
|    Project|
|  Gutenberg|
|      EBook|
|         of|
|        The|
|   Complete|
|      Works|
|         of|
|    William|
|Shakespeare|
|         by|
|    William|
|Shakespeare|
|           |
|       This|
|      eBook|
|         is|
|        for|
|        the|
+-----------+
only showing top 20 rows

+---------------+------+
|starting_letter| count|
+---------------+------+
|              _|     2|
|              0|    10|
|              X|    14|
|              7|    21|
|              8|    21|
|              6|    26|
|              9|    32|
|              5|    39|
|              4|    49|
|              3|    69|
|              Z|    79|
|              2|   325|
|              1|   917|
|              Q|  2388|
|              J|  3372|
|              V|  5801|
|              U|  9230|
|              K|  9535|
|              R| 15234|
|              E| 20409|
|              G| 21167|
|              Y| 25926|
|            

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!